# Generating the Two DOF Example System
This notebook defines the properties for the two DOF example system that will be used in the basic examples. A graphic of the different systems is shown below, where system A is on the left, system B is on the right, and the coupled system is in the center. 

![Two_DOF_System](./Images/TwoMassSystem.svg)

The data in this notebook will be written to SDynPy System and TransferFunctionArray objects to simplify the bookkeeping for various computations. These SDynPy objects will be saved to the root folder of the examples and will be re-loaded into the notebooks for the different examples, as necessary. 

In [1]:
import numpy as np
import sdynpy as sdpy

## System A

In [2]:
m1 = 1 
m2 = 1 
Ma = np.array([[m1, 0],
               [0,  m2]])

k1 = 10000
k2 = 20000
Ka = np.array([[k1+k2, -k2],
               [-k2,    k2]])

c1 = 1
c2 = 2
Ca = np.array([[c1+c2, -c2],
               [-c2,    c2]])

# SDynPy objects require a DOF list for the nodes in the models of the systems
dof_a = sdpy.coordinate_array(node=[1,2], direction=1)

# Defining the SDynPy system to simplify the computations
system_a = sdpy.System(dof_a, Ma, Ka, Ca)

### System A Modal Properties and FRFs
The frequency vector for the FRFs was created to fully observe the modes of both example systems.

```{note}
The `displacement_derivative` kwarg is set to two so the resulting FRFs are in accelerance format.
```

In [3]:
# Computes a square FRF matrix by default
freq = np.arange(601)*0.1
system_a_frfs = system_a.frequency_response(frequencies=freq, displacement_derivative=2)
print(system_a.eigensolution())

   Index,  Frequency,    Damping,     # DoFs
    (0,),    10.5385,    0.3311%,          2
    (1,),    33.9920,    1.0679%,          2



## System B

In [4]:
m3 = 1 
m4 = 1 
Mb = np.array([[m3, 0],
               [0,  m4]])

k3 = 30000
k4 = 40000
Kb = np.array([[k3, -k3],
               [-k3, k3+k4]])

c3 = 1
c4 = 2
Cb = np.array([[c3, -c3],
               [-c3, c3+c4]])

dof_b = sdpy.coordinate_array(node=[3,4], direction=1)
system_b = sdpy.System(dof_b, Mb, Kb, Cb)

### System B Modal Properties and FRFs

In [5]:
system_b_frfs = system_b.frequency_response(frequencies=freq, displacement_derivative=2)
print(system_b.eigensolution())

   Index,  Frequency,    Damping,     # DoFs
    (0,),    18.7941,    0.2597%,          2
    (1,),    46.6885,    0.5772%,          2



## Coupled System
This section creates the coupled system to provide validation data for the different FBS examples. 

In [6]:
M_coupled = np.array([[m1, 0,     0],
                      [0,  m2+m3, 0],
                      [0,  0,     m4]])

K_coupled = np.array([[k1+k2, -k2,     0],
                      [-k2,    k2+k3, -k3],
                      [0,     -k3,     k3+k4]])

C_coupled = np.array([[c1+c2, -c2,     0],
                      [-c2,    c2+c3, -c3],
                      [0,     -c3,     c3+c4]])

# The coupling DOF is labeled as 2, for simplicity
dof_coupled = sdpy.coordinate_array(node=[1,2,4], direction=1)
coupled_system = sdpy.System(dof_coupled, M_coupled, K_coupled, C_coupled)

### Coupled System Modal Properties and FRFs

In [7]:
# Computes a square FRF matrix by default
coupled_system_frfs = coupled_system.frequency_response(frequencies=freq, displacement_derivative=2)
print(coupled_system.eigensolution())

   Index,  Frequency,    Damping,     # DoFs
    (0,),    14.6067,    0.2828%,          3
    (1,),    30.8468,    0.9240%,          3
    (2,),    44.7371,    0.6047%,          3



## Saving the Data for Future Use
This section saves the SDynPy System and TransferFunctionArray objects to the root folder of the book for future use by other notebooks for the different example problems. 

In [8]:
system_a.save('./example_systems/system_a.npz')
system_a_frfs.save('./example_frfs/system_a_frfs.npz')

system_b.save('./example_systems/system_b.npz')
system_b_frfs.save('./example_frfs/system_b_frfs.npz')

coupled_system.save('./example_systems/coupled_system.npz')
coupled_system_frfs.save('./example_frfs/coupled_system_frfs.npz')